#### 作业3代码部分-杨思涵-2023110085 

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
#读取数据
data=[]
with open('browsing.txt') as f:
    for line in f:
        data.append(line.strip().split())
for i in range(0,100):
    print(data[i])

['FRO11987', 'ELE17451', 'ELE89019', 'SNA90258', 'GRO99222']
['GRO99222', 'GRO12298', 'FRO12685', 'ELE91550', 'SNA11465', 'ELE26917', 'ELE52966', 'FRO90334', 'SNA30755', 'ELE17451', 'FRO84225', 'SNA80192']
['ELE17451', 'GRO73461', 'DAI22896', 'SNA99873', 'FRO86643']
['ELE17451', 'ELE37798', 'FRO86643', 'GRO56989', 'ELE23393', 'SNA11465']
['ELE17451', 'SNA69641', 'FRO86643', 'FRO78087', 'SNA11465', 'GRO39357', 'ELE28573', 'ELE11375', 'DAI54444']
['ELE17451', 'GRO73461', 'DAI22896', 'SNA99873', 'FRO18919', 'DAI50921', 'SNA80192', 'GRO75578']
['ELE17451', 'ELE59935', 'FRO18919', 'ELE23393', 'SNA80192', 'SNA85662', 'SNA91554', 'DAI22177']
['ELE17451', 'SNA69641', 'FRO18919', 'SNA90258', 'ELE28573', 'ELE11375', 'DAI14125', 'FRO78087']
['ELE17451', 'GRO73461', 'DAI22896', 'SNA80192', 'SNA85662', 'SNA90258', 'DAI46755', 'FRO81176', 'ELE66810', 'DAI49199', 'DAI91535', 'GRO94758', 'ELE94711', 'DAI22177']
['ELE17451', 'SNA69641', 'DAI91535', 'GRO94758', 'GRO99222', 'FRO76833', 'FRO81176', 'SNA80

##### d. 识别那些支持度至少为 100 的产品对 (X, Y )。对于所有满足条件的产品对，计算相应关联规则（X ⇒ Y, Y ⇒ X）的置信度分数。按照置信度分数的降序对规则进行排序，并列出排序后的前 5 条规则

In [26]:
def A_Priori_First(data):  #A_priori第一次扫描
    dic = {}
    for record in data: 
        for item in record:  
            if item not in dic:  
                dic[item] = 1  
            else:
                dic[item] += 1 
    min_count=100
    frequent_items ={}
    for key, value in dic.items():  
        if dic[key] >= min_count:
            frequent_items[key]=value          
    return frequent_items

In [27]:
from itertools import combinations
def A_Priori_Second(data,Frequent_items):
    frequent_keys = list(Frequent_items.keys())
    candidate_pairs = list(combinations(frequent_keys, 2))
    pair_support = {}  # 候选项对支持度
    for pair in candidate_pairs:
        pair_support[pair] = 0 
    for record in data:
        for pair in candidate_pairs:
            if pair[0] in record and pair[1] in record:
                pair_support[pair] += 1
    frequent_pairs={}
    min_support = 100
    for pair in pair_support:
        if pair_support[pair] >= min_support:
            frequent_pairs[pair] = pair_support[pair]
    return frequent_pairs
            

In [28]:
frequent_items=A_Priori_First(data)
frequent_pairs=A_Priori_Second(data,frequent_items)

In [29]:
rules = []
for (X, Y), support_xy in frequent_pairs.items():
    conf_x_to_y = support_xy / frequent_items[X]
    conf_y_to_x = support_xy / frequent_items[Y]
    rules.append((X, Y, conf_x_to_y))
    rules.append((Y, X, conf_y_to_x))
rules.sort(key=lambda x: x[2], reverse=True)
print("Top 5 Confidence Rules:")
for i, (X, Y, conf) in enumerate(rules[:5], 1):
    print(f"{i}. {X} → {Y}: {conf:.4f}")

Top 5 Confidence Rules:
1. DAI93865 → FRO40251: 1.0000
2. GRO85051 → FRO40251: 0.9992
3. GRO38636 → FRO40251: 0.9907
4. ELE12951 → FRO40251: 0.9906
5. DAI88079 → FRO40251: 0.9867


##### e. 识别那些支持度至少为 100 的产品三元组 (X, Y, Z)。对于所有满足条件的产品三元组，计算相应关联规则（(X, Y ) ⇒ Z,(X, Z) ⇒ Y,(Y, Z) ⇒ X）的置信度分数。按照置信度分数的降序对规则进行排序，并列出排序后的前 5 条规则。

In [32]:
frequent_items_keys = list(frequent_items.keys())
candidate_triples = []
for triple in combinations(frequent_items_keys, 3):
    sub_pairs = [(triple[0], triple[1]),(triple[0], triple[2]),(triple[1], triple[2])]
    if all(pair in frequent_pairs for pair in sub_pairs):
        candidate_triples.append(triple)

triple_support = {}
for triple in candidate_triples:
    triple_support[triple] = 0

for record in data:
    for triple in candidate_triples:
        if all(item in record for item in triple):
            triple_support[triple] += 1

min_support = 100
frequent_triples = {
    triple: count for triple, count in triple_support.items() if count >= min_support
}

rules = []

for (X, Y, Z), support_xyz in frequent_triples.items():
    pair_xy = tuple([X, Y])
    pair_xz = tuple([X, Z])
    pair_yz = tuple([Y, Z])

    conf_xy_to_z = support_xyz / frequent_pairs[pair_xy]
    conf_xz_to_y = support_xyz / frequent_pairs[pair_xz]
    conf_yz_to_x = support_xyz / frequent_pairs[pair_yz]

    rules.append((X, Y, Z, conf_xy_to_z))
    rules.append((X, Z, Y, conf_xz_to_y))
    rules.append((Y, Z, X, conf_yz_to_x))

rules.sort(key=lambda x: x[3], reverse=True)

print("Top 5 Confidence Rules (3-itemsets):")
for i, (X, Y, Z, conf) in enumerate(rules[:5], 1):
    print(f"{i}. ({X},{Y}) → {Z}: {conf:.4f}")
print('\n第六位之后仍有关联规则置信度为1，但是由于题目要求这里不予展示。')

Top 5 Confidence Rules (3-itemsets):
1. (ELE17451,GRO85051) → FRO40251: 1.0000
2. (ELE26917,GRO85051) → FRO40251: 1.0000
3. (GRO73461,GRO85051) → FRO40251: 1.0000
4. (DAI55911,GRO85051) → FRO40251: 1.0000
5. (DAI62779,DAI88079) → FRO40251: 1.0000

第六位之后仍有关联规则置信度为1，但是由于题目要求这里不予展示。
